In [ ]:
import sympy
import numpy as np
import matplotlib.pyplot as plt
import dill
dill.settings["recurse"] = True
from sympy import Rational as R
from tqdm.notebook import tqdm 
import plotly

## Define necessary symbols

In [ ]:
x_hat, y_hat = sympy.symbols(r"\hat{x} \hat{y}")

x_0, x_1, x_2 = sympy.symbols("x_0:3")
y_0, y_1, y_2 = sympy.symbols("y_0:3")

In [ ]:
s, t = sympy.symbols("s t")
s_1, s_2 = sympy.symbols("s_1 s_2")

In [ ]:
monomial_basis = sympy.Matrix(
    [
        1,
        x_hat,
        y_hat,
    ]
)

V = sympy.zeros(3, 3)

for dofidx, basis in enumerate(monomial_basis):

    V[dofidx, 0] = basis.subs({x_hat: 0, y_hat: 0})
    V[dofidx, 1] = basis.subs({x_hat: 1, y_hat: 0})
    V[dofidx, 2] = basis.subs({x_hat: 0, y_hat: 1})

mapping_basis = V.inv() @ monomial_basis

global_x = (
      mapping_basis[0] * x_0
    + mapping_basis[1] * x_1
    + mapping_basis[2] * x_2
)
global_y = (
      mapping_basis[0] * y_0
    + mapping_basis[1] * y_1
    + mapping_basis[2] * y_2
)

mapping_function = sympy.Matrix([global_x, global_y])

In [ ]:
f_1 = sympy.Function("f_1")(s)
f_2 = sympy.Function("f_2")(s)

In [ ]:
chi = sympy.Matrix([f_1, f_2])

In [ ]:
temp_param = s_1 + t*(s_2 - s_1)

In [ ]:
# phi = chi.subs({
#     f_1 : sympy.cos(s),
#     f_2 : sympy.sin(s),
#     })

In [ ]:
phi = chi.subs({s: temp_param})

In [ ]:
L = sympy.Matrix([
                    x_1 + t*(x_2 - x_1),
                    y_1 + t*(y_2 - y_1),
])

In [ ]:
Phi = ((phi - L) / (1 - t)).subs({t: y_hat})

In [ ]:
X = mapping_function + x_hat * Phi

In [144]:
X.subs({x_hat:1/2, y_hat:1/2})

Matrix([
[1.0*f_1(0.5*s_1 + 0.5*s_2)],
[1.0*f_2(0.5*s_1 + 0.5*s_2)]])

In [ ]:
sympy.lambdify([
    x_hat, y_hat,
    x_0, y_0,
    x_1, y_1,
    x_2, y_2,
    s_1, s_2
], X)(*np.random.randn(10))

In [ ]:
# Causes det == 0 when t_hat == 0, why?
# X = sympy.Matrix([
#             sympy.Piecewise((X[0, 0], y_hat > 0), (sympy.limit(X[0, 0], y_hat, 0), True)),
#             sympy.Piecewise((X[1, 0], y_hat > 0), (sympy.limit(X[1, 0], y_hat, 0), True)),            
# ])

In [ ]:
det = X.jacobian([x_hat, y_hat]).det()

## Test the result on an example

In [ ]:
points = np.array([    
    [0, 0],
    [1, 0],
    [0, 1],
]).astype(float)

In [ ]:
grid = np.linspace(0, 1, 30)
grid_x, grid_y = np.meshgrid(grid, grid, indexing="ij")
mask = (grid_y <= 1 - grid_x)
grid_x = grid_x[mask]
grid_y = grid_y[mask]

In [ ]:
px, py = sympy.lambdify([
    x_hat, y_hat,
    x_0, y_0,
    x_1, y_1,
    x_2, y_2,
    s_1, s_2
], X)(
        grid_x, grid_y, 
        points[0, 0], points[0, 1], 
        points[1, 0], points[1, 1], 
        points[2, 0], points[2, 1], 
        0, np.pi/2).squeeze()

pz = sympy.lambdify([
    x_hat, y_hat,
    x_0, y_0,
    x_1, y_1,
    x_2, y_2,
    s_1, s_2
], det)(
        grid_x, grid_y, 
        points[0, 0], points[0, 1], 
        points[1, 0], points[1, 1], 
        points[2, 0], points[2, 1], 
        0, np.pi/2).squeeze()

In [ ]:
plt.scatter(px, py)
plt.axis("equal")
plt.xlim([0, 1])
plt.ylim([0, 1])

In [ ]:
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=px[~np.isnan(px)], 
                                y=py[~np.isnan(px)],
                                z=pz[~np.isnan(px)],
                                intensity=pz[~np.isnan(px)]
                                )        
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()

In [ ]:
import gmsh
import math

# Initialize Gmsh
gmsh.initialize()
gmsh.model.add("parametric_spline")

# Define the start and end points
start_point = gmsh.model.geo.addPoint(0, 0, 0)
end_point = gmsh.model.geo.addPoint(1, 0, 0)

# Define the control points for the spline
control_points = []

N = 10
for t in range(N+1):
    x = math.sin(t / N * 2 * math.pi)
    y = math.cos(t / N * 2 * math.pi)
    z = 0
    control_points.append(gmsh.model.geo.addPoint(x, y, z))

# Create the spline
spline = gmsh.model.geo.addSpline(control_points)

# Synchronize necessary before meshing
gmsh.model.geo.synchronize()

# Generate mesh
gmsh.model.mesh.generate(1)

# Save and finalize
# gmsh.write("parametric_spline.msh")
gmsh.fltk.run()